<a href="https://colab.research.google.com/github/heros-lab/colaboratory/blob/master/verification_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
work_path = "/content/drive/My Drive/Colab Notebooks"


In [ ]:
def create_model(x_dim, y_dim, units):
    alpha=1.00e-3
    act_function = "tanh"
    init_method  = "glorot_uniform"
    model = Sequential()
    model.add(Dense(
            input_dim=x_dim, units=units[0],
            activation=act_function, kernel_initializer=init_method))
    for i in range(len(units) - 1):
        model.add(Dense(
            input_dim=units[i], units=units[i+1],
            activation=act_function, kernel_initializer=init_method))
    model.add(Dense(input_dim=units[-1], units=y_dim))
    model.compile(loss="mse", optimizer=Adam(lr=alpha))
    return model


In [ ]:
x_index = [0,1,2,3,4,5,6] # model_n1, n2
#x_index = [0,3,4,6] # model_s1
#x_index = [1,2,5,6] # model_s2

#y_index = [0]
y_index = [2]

learn_name = "ms2a"
test_name  = "ms1a"

learn_x = np.load(f"{work_path}/learn/{learn_name}_x.npy")[:, x_index]
learn_y = np.load(f"{work_path}/learn/{learn_name}_y.npy")[:, y_index]

test_x = np.load(f"{work_path}/learn/{test_name}_x.npy")[:, x_index]
test_y = np.load(f"{work_path}/learn/{test_name}_y.npy")[:, y_index]


In [ ]:
epochs = 200
num_batch = 256
num_sample = 101

num_units = [200, 200]

#num_units = [19, 17]  # model_n1
#num_units = [3, 15]   # model_s1

for i in range(num_sample):
    clear_session()
    print(f"\rsampling: #{i:3} / {num_sample-1}", end="")
    
    model = create_model(learn_x.shape[1], 1, num_units)
    loss = model.fit(learn_x, learn_y, batch_size=num_batch, epochs=epochs, verbose=0)
    resp = model.predict(test_x)[:, 0]
    error = test_y[0] - resp

    if i == 0:
        df_loss = pd.DataFrame({f"#{i}": loss.history["loss"]})
        df_resp = pd.DataFrame({f"#{i}": resp})
        df_error = pd.DataFrame({f"#{i}": error})
    else:
        df_loss[f"#{i}"] = loss.history["loss"]
        df_resp[f"#{i}"] = resp
        df_error[f"#{i}"] = error


In [ ]:
model_tag = "s1"

df_loss.to_csv(f"{work_path}/vrf_{model_tag}_loss.csv")
df_resp.to_csv(f"{work_path}/vrf_{model_tag}_resp.csv")
df_error.to_csv(f"{work_path}/vrf_{model_tag}_error.csv")